## Import the labrary and define the function

In [ ]:
import glob
import csv
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from  sklearn.utils import shuffle
from tensorflow.contrib.layers import flatten

In [ ]:
## define a function to translate the string data into a float list

def str2array(string):    
    together = ''
    for i in string:
        if(i != '['):
            together += i
    
    first = together.split("],")
    ret = []
    
    for i in range(len(first)):
        if(i==len(first)-1):
            second = ''
            for j in first[i]:
                if(j!=']'):
                    second += j
            second = second.split(",")
        else:
            second = first[i].split(",")
        ret.append([float(second[0]), float(second[1])])
    return ret

## define a function for exploring the maximum and minimum length of the track data

def find_maxmin(data):
    maximum = 0
    minimum = 99999
    for track in data:
        if(len(track) > maximum):
            maximum = len(track)
        if(len(track) < minimum):
            minimum = len(track)
    return maximum, minimum

## def a functino for exploring the distribution of the track length

def count(data, number = 10, maximum = 4000):
    count_num = []
    part = maximum//number
    level = part*np.array(range(1+number))
    print("level is:", level)
    
    for i in range(number):
        count_num.append(0)
    
    for line in data:
        for i in range(len(level)-1):
            if((len(line) < level[i+1]) & (len(line) >= level[i])):
                count_num[i] += 1
                continue
    return count_num

## define a function for exploring the number of the track which is above a value

def above(data, level):
    count = 0
    for line in data:
        if(len(line)>=level):
            count += 1            
    return count

## define a function for selecting track data

def select_data(track, place):
    ret = []
    for i in place:
        ret.append(track[i])
    return ret

def pick_data(data, data_length):
    ret_train = []
    ret_label = []
    n = 0
    for i in range(len(data)):
        if(len(data[i]) > data_length):
            split = len(data[i])//data_length
            ret_train.append([])
            for j in range(data_length):
                ret_train[n].append(data[i][split*j])
            ret_label.append(data[i][len(data[i])-1])
            n += 1
    return ret_train, ret_label

## Import the data

In [ ]:
with open('../train_data.pickle', 'rb') as file:
    data_dict = pickle.load(file)
    train_data = data_dict['train_data10']
    train_label = data_dict['train_label10']    

In [ ]:
print(train_data[0])
print()
print(train_label[:100])

In [ ]:
## divide the area into n parts, only predict the area for the destination
max_x = -99999.;
max_y = -99999.;
min_x = 99999.;
min_y = 99999.;

for point in train_label:
    if max_x < point[0]: max_x = point[0]
    if max_y < point[1]: max_y = point[1]
    if min_x > point[0]: min_x = point[0]
    if min_y > point[1]: min_y = point[1]

In [ ]:
print(min_x, min_y, max_x, max_y)

In [ ]:
div_len = 0.16
area = []
labels = []
x = min_x
y = min_y
i = 0
while(x < max_x + div_len):
    y = min_y
    while(y < max_y + div_len):
        area.append([x, y])
        y += div_len
    i += 1
    x += div_len
    
print("Together %d areas" % (len(area) * len(area[0])))

In [ ]:
print(area[:100])

In [ ]:
labels = []
for point in train_label:
    find = False    
    for i in range(len(area) - 1):
        if((point[0] < area[i][0]) & (point[0] < area[i][1])):
            find = True 
            labels.append(i)
            break
    if not find:
        print("Error at i = %d" % i)
        print("Point:", point)
        break

In [ ]:
print(len(labels), len(train_data), len(train_label))

In [ ]:
## count how many times a label shows up
count = []
showup = []
for index in labels:
    add = True
    for i in range(len(showup)):
        if(showup[i] == index):
            add = False
            count[i] += 1
    if(add): 
        showup.append(index)
        count.append(1)

In [ ]:
for i in range(len(showup)):
    print("{0:<5}".format(showup[i]), ':', count[i])

## Split data into train, validation and test data

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(train_data, labels, test_size = 0.3, random_state = 42)

In [ ]:
## save the data into a pickle file
# need to save: data, area, labels_for_train
data_dict = {'x_train': x_train, 'x_test': x_test, 'y_train': y_train, 'y_test': y_test, "area": area,
            "labels": labels}
with open('./data/train_for_deepstructure_area.pickle','wb') as file:
    pickle.dump(data_dict, file)

In [ ]:
with open('./data/train_for_deepstructure_area.pickle', 'rb') as file:
    data_dict = pickle.load(file)
    x_train = data_dict['x_train']
    x_test = data_dict['x_test']    
    y_train = data_dict['y_train']
    y_test = data_dict['y_test']
    area = data_dict['area']
    labels = data_dict['labels']

# Build an easy network

In [ ]:
x = tf.placeholder(tf.float32,(None, 20))

# Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
mu = 0
sigma = 0.1

# Layer 1: Input = 20. Output = 3600.
fc1_W = tf.Variable(tf.truncated_normal(shape=(20, 1000), mean = mu, stddev = sigma))
fc1_b = tf.Variable(tf.zeros(1000))
fc1_lin   = tf.matmul(x, fc1_W) + fc1_b

# Activation.
fc1 = tf.nn.relu(fc1_lin)

# Layer 2: Input = 1000. Output = 3600.
fc2_W = tf.Variable(tf.truncated_normal(shape=(1000, 3600), mean = mu, stddev = sigma))
fc2_b = tf.Variable(tf.zeros(3600))
logits   = tf.matmul(fc1, fc2_W) + fc2_b

In [ ]:
y = tf.placeholder(tf.int32,(None))
one_hot_y = tf.one_hot(y,3600)

learning_rate = 0.001

cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels = one_hot_y, logits = logits)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
training_operation = optimizer.minimize(loss_operation)

saver = tf.train.Saver()

In [ ]:
# tensorflow parameters
epoches = 50
batch_size = 128

In [ ]:
## Evaluation accuracy

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, batch_size):
        batch_x, batch_y = X_data[offset:offset+batch_size], y_data[offset:offset+batch_size]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

In [ ]:
# training
# when the accuracy is high, I want to make the learning rate smaller.
n_train = len(x_train)
with tf.device('/cpu:0'):
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        print('Training ......')
        print()
        max_acc = 0

        for i in range(epoches):
            X_train,y_train = shuffle(x_train,y_train) # shuffle the data
            for offset in range(0,n_train,batch_size):
                end = offset + batch_size
                batch_x, batch_y = X_train[offset:end],y_train[offset:end]
                sess.run(training_operation,feed_dict = {x:batch_x, y:batch_y})

            validation_accuracy = evaluate(x_test, y_test)
            if(max_acc <= validation_accuracy):
                max_acc = validation_accuracy
                saver.save(sess, './result')
                print("Model saved with accuracy:", max_acc)
            if i%5 == 0:
                print("EPOCH {} ...".format(i+1))
                print("Validation Accuracy = {:.3f}".format(validation_accuracy))
    #             print('Can be better, continue training...')
                print()

        print("maximal accuracy is:", max_acc)
        print("End training")

## only 1133

In [ ]:
with open('../trackdata1133.pickle', 'rb') as file:
    data_dict = pickle.load(file)
    x_train = data_dict['x_train']
    x_test = data_dict['x_test']
    y_train = data_dict['y_train']
    y_test = data_dict['y_test']

In [ ]:
x = tf.placeholder(tf.float32,(None, 20))

# Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
mu = 0
sigma = 0.1

# Layer 1: Input = 20. Output = 100.
fc1_W = tf.Variable(tf.truncated_normal(shape=(20, 100), mean = mu, stddev = sigma))
fc1_b = tf.Variable(tf.zeros(100))
fc1_lin   = tf.matmul(x, fc1_W) + fc1_b

# Activation.
fc1 = tf.nn.relu(fc1_lin)

# Layer 2: Input = 100. Output = 900.
fc2_W = tf.Variable(tf.truncated_normal(shape=(100, 900), mean = mu, stddev = sigma))
fc2_b = tf.Variable(tf.zeros(900))
logits   = tf.matmul(fc1, fc2_W) + fc2_b

In [ ]:
y = tf.placeholder(tf.int32,(None))
one_hot_y = tf.one_hot(y,900)

learning_rate = 0.001

cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels = one_hot_y, logits = y)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
training_operation = optimizer.minimize(loss_operation)

saver = tf.train.Saver()

In [ ]:
# tensorflow parameters
epoches = 50
batch_size = 128

In [ ]:
## Evaluation accuracy

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, batch_size):
        batch_x, batch_y = X_data[offset:offset+batch_size], y_data[offset:offset+batch_size]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

In [ ]:
n_train = len(x_train)
## with tf.device('/cpu:0'):
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print('Training ......')
    print()
    max_acc = 0

    for i in range(epoches):
        X_train,y_train = shuffle(x_train,y_train) # shuffle the data
        for offset in range(0,n_train,batch_size):
            end = offset + batch_size
            batch_x, batch_y = X_train[offset:end],y_train[offset:end]
            sess.run(training_operation,feed_dict = {x:batch_x, y:batch_y})

        validation_accuracy = evaluate(x_test, y_test)
        if(max_acc <= validation_accuracy):
            max_acc = validation_accuracy
            saver.save(sess, './result')
            print("Model saved with accuracy:", max_acc)
        if i%5 == 0:
            print("EPOCH {} ...".format(i+1))
            print("Validation Accuracy = {:.3f}".format(validation_accuracy))
#             print('Can be better, continue training...')
            print()

    print("maximal accuracy is:", max_acc)
    print("End training")

## RNN

In [3]:
## rnn method

## rnn parameters
n_inputs = 20
n_steps = 1
n_hidden = 900
n_classes = 900

x = tf.placeholder(tf.float32,(None, 20))
y = tf.placeholder(tf.int32, None)
size = tf.placeholder(tf.int32)
one_hot_y = tf.one_hot(y,900)

weights = {
    
    'in': tf.Variable(tf.random_normal([n_inputs, n_hidden])),
    
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}

biases = {
    
    'in': tf.Variable(tf.constant(0.1, shape=[n_hidden, ])),
    
    'out': tf.Variable(tf.constant(0.1, shape=[n_classes, ]))
}

In [4]:
# tensorflow parameters
epoches = 1000
batch_size = 128
learning_rate = 0.001
saver = tf.train.Saver()

In [5]:
def RNN(X, weights, biases):
    
    X = tf.reshape(X, [-1, n_inputs])
    X_in = tf.matmul(X, weights['in']) + biases['in']
    X_in = tf.reshape(X_in, [-1, n_steps, n_hidden])
    
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
    init_state = lstm_cell.zero_state(batch_size, dtype=tf.float32)
    
    outputs, final_state = tf.nn.dynamic_rnn(lstm_cell, X_in, initial_state=init_state, time_major=False)
    
    results = tf.matmul(final_state[1], weights['out']) + biases['out']
    
    return results

In [6]:
pred = RNN(x, weights, biases)

In [7]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = pred, labels = one_hot_y))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [8]:
train_op = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [9]:
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples-78, batch_size):
        batch_x, batch_y = X_data[offset:offset+batch_size], y_data[offset:offset+batch_size]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

In [ ]:
n_train = len(x_train)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print('Training ......')
    print()
    max_acc = 0
    
    for i in range(epoches):
        X_train,y_train = shuffle(x_train,y_train)
        for offset in range(0,n_train,batch_size):
            end = offset + batch_size
            batch_x, batch_y = X_train[offset:end],y_train[offset:end]
#             size = len(batch_x)
#             batch_x = batch_x.reshape([size, n_inputs])
            if(len(batch_x) != batch_size): continue
            sess.run(train_op,feed_dict = {x:batch_x, y:batch_y})
            
        
        validation_accuracy = evaluate(x_test, y_test)
        
        if(max_acc < validation_accuracy):
            max_acc = validation_accuracy
            saver.save(sess, './result')
            print("Model saved with accuracy:", max_acc)
        if i%5 == 0:
            print("EPOCH {} ...".format(i+1))
            print("Validation Accuracy = {:.3f}".format(validation_accuracy))
#             print('Can be better, continue training...')
            print()

    print("maximal accuracy is:", max_acc)
    print("End training")